In [1]:
import pandas as pd

In [2]:
payments = pd.read_csv("old/sidooh_payments.csv")
vouchers = pd.read_csv("old/sidooh_vouchers.csv")

stk_requests = pd.read_csv("old/sidooh_mpesa_stk_requests.csv")
stk_callbacks = pd.read_csv("old/sidooh_mpesa_stk_callbacks.csv")

c2b_callbacks = pd.read_csv("old/sidooh_mpesa_c2b_callbacks.csv")

bulk_requests = pd.read_csv("old/sidooh_mpesa_bulk_payment_requests.csv")
bulk_responses = pd.read_csv("old/sidooh_mpesa_bulk_payment_responses.csv")
b2c_result_parameters = pd.read_csv("old/sidooh_mpesa_b2c_result_parameters.csv")

In [3]:
payments_new = payments.copy()
vouchers_new = vouchers.copy()

stk_requests_new = stk_requests.copy()
stk_callbacks_new = stk_callbacks.copy()

c2b_callbacks_new = c2b_callbacks.copy()

bulk_requests_new = bulk_requests.copy()
bulk_responses_new = bulk_responses.copy()
b2c_result_parameters_new = b2c_result_parameters.copy()

In [4]:
payments_new

,id,payable_id,payable_type,amount,status,type,subtype,payment_id,start_date,created_at,updated_at
0,1,1,App\Models\Transaction,20,Complete,MPESA,STK,1,2021-02-01 16:38:25,2021-02-01 17:10:23,2021-02-01 17:13:57
1,2,2,App\Models\Transaction,50,Complete,MPESA,STK,2,2021-02-01 16:38:25,2021-02-01 17:11:04,2021-02-01 17:11:47
2,3,3,App\Models\Transaction,475,Complete,SIDOOH,VOUCHER,1,2021-02-01 16:38:25,2021-02-01 17:17:15,2021-02-01 17:17:15
3,4,4,App\Models\Transaction,8775,Failed,MPESA,STK,3,2021-02-01 16:38:25,2021-02-01 17:20:44,2021-02-01 17:20:44
4,5,5,App\Models\Transaction,20,Failed,MPESA,STK,4,2021-02-01 16:38:25,2021-02-01 17:22:49,2021-02-01 17:22:49
...,...,...,...,...,...,...,...,...,...,...,...
12985,12986,12993,App\Models\Transaction,100,Complete,MPESA,STK,12183,2021-02-01 16:38:25,2022-07-21 18:26:49,2022-07-21 18:27:01
12986,12987,12994,App\Models\Transaction,50,Complete,MPESA,STK,12184,2021-02-01 16:38:25,2022-07-21 18:51:13,2022-07-21 18:51:24
12987,12988,12995,App\Models\Transaction,50,Failed,MPESA,STK,12185,2021-02-01 16:38:25,2022-07-21 18:53:08,2022-07-21 18:53:19
12988,12989,12996,App\Models\Transaction,80,Complete,MPESA,STK,12186,2021-02-01 16:38:25,2022-07-21 18:55:05,2022-07-21 18:55:19


In [5]:
payments_new['description'] = ''
payments_new['reference'] = ''
payments_new['provider_id'] = payments_new['payment_id']

del payments_new['payable_type']
del payments_new['start_date']
del payments_new['payment_id']

def statusTransform(val):
    val = val.upper()
    
    if (val in ['COMPLETE', 'PAID']):
        val = 'COMPLETED'
        
    if (val == 'CANCELLED'):
        val = 'FAILED'
    
    return val


payments_new['status'] = payments['status'].apply(statusTransform)

In [6]:
payments_new

,id,payable_id,amount,status,type,subtype,created_at,updated_at,description,reference,provider_id
0,1,1,20,COMPLETED,MPESA,STK,2021-02-01 17:10:23,2021-02-01 17:13:57,,,1
1,2,2,50,COMPLETED,MPESA,STK,2021-02-01 17:11:04,2021-02-01 17:11:47,,,2
2,3,3,475,COMPLETED,SIDOOH,VOUCHER,2021-02-01 17:17:15,2021-02-01 17:17:15,,,1
3,4,4,8775,FAILED,MPESA,STK,2021-02-01 17:20:44,2021-02-01 17:20:44,,,3
4,5,5,20,FAILED,MPESA,STK,2021-02-01 17:22:49,2021-02-01 17:22:49,,,4
...,...,...,...,...,...,...,...,...,...,...,...
12985,12986,12993,100,COMPLETED,MPESA,STK,2022-07-21 18:26:49,2022-07-21 18:27:01,,,12183
12986,12987,12994,50,COMPLETED,MPESA,STK,2022-07-21 18:51:13,2022-07-21 18:51:24,,,12184
12987,12988,12995,50,FAILED,MPESA,STK,2022-07-21 18:53:08,2022-07-21 18:53:19,,,12185
12988,12989,12996,80,COMPLETED,MPESA,STK,2022-07-21 18:55:05,2022-07-21 18:55:19,,,12186


In [7]:
product_payments = payments_new.copy()


selector_d = {
    'id': 'payment_id',
    'amount': 'amount',
    'type': 'type',
    'subtype': 'subtype',
    'status': 'status',
    'reference': 'extra',
    'transaction_id': 'payable_id',
    'created_at': 'created_at',
    'updated_at': 'updated_at'
}

product_payments = product_payments.rename(columns=selector_d)[selector_d.values()]

In [8]:
# del payments_new['payable_id']

In [9]:
vouchers_new

,id,in,out,created_at,updated_at,account_id
0,1,3161.0,2110.0,2021-02-01 17:10:23,2022-06-23 06:32:15,1
1,2,820.0,820.0,2021-02-01 17:11:04,2022-06-07 17:42:14,2
2,3,0.0,0.0,2021-02-01 17:22:49,2021-02-01 17:22:49,3
3,4,1144.0,1144.0,2021-02-01 18:17:12,2022-05-06 09:37:37,4
4,5,100.0,100.0,2021-02-01 18:35:16,2021-03-31 14:43:09,5
...,...,...,...,...,...,...
819,820,0.0,0.0,2022-07-04 04:15:12,2022-07-04 04:15:12,820
820,821,0.0,0.0,2022-07-05 04:06:40,2022-07-05 04:06:40,821
821,822,0.0,0.0,2022-07-07 18:07:34,2022-07-07 18:07:34,822
822,823,0.0,0.0,2022-07-09 17:58:05,2022-07-09 17:58:05,823


In [10]:
vouchers_new['type'] = 'SIDOOH'
vouchers_new['balance'] = vouchers_new['in'] - vouchers_new['out']

In [11]:
vouchers_new

,id,in,out,created_at,updated_at,account_id,type,balance
0,1,3161.0,2110.0,2021-02-01 17:10:23,2022-06-23 06:32:15,1,SIDOOH,1051.0
1,2,820.0,820.0,2021-02-01 17:11:04,2022-06-07 17:42:14,2,SIDOOH,0.0
2,3,0.0,0.0,2021-02-01 17:22:49,2021-02-01 17:22:49,3,SIDOOH,0.0
3,4,1144.0,1144.0,2021-02-01 18:17:12,2022-05-06 09:37:37,4,SIDOOH,0.0
4,5,100.0,100.0,2021-02-01 18:35:16,2021-03-31 14:43:09,5,SIDOOH,0.0
...,...,...,...,...,...,...,...,...
819,820,0.0,0.0,2022-07-04 04:15:12,2022-07-04 04:15:12,820,SIDOOH,0.0
820,821,0.0,0.0,2022-07-05 04:06:40,2022-07-05 04:06:40,821,SIDOOH,0.0
821,822,0.0,0.0,2022-07-07 18:07:34,2022-07-07 18:07:34,822,SIDOOH,0.0
822,823,0.0,0.0,2022-07-09 17:58:05,2022-07-09 17:58:05,823,SIDOOH,0.0


In [12]:
voucher_trans = vouchers_new.copy()
voucher_trans[['created_at', 'updated_at']] = pd.to_datetime('now')
voucher_trans['voucher_id'] = voucher_trans['id']

del voucher_trans['balance']
del voucher_trans['account_id']
del voucher_trans['id']

voucher_transactions_in = voucher_trans.copy()

voucher_transactions_in['type'] = 'CREDIT'
voucher_transactions_in['amount'] = voucher_transactions_in['in']
voucher_transactions_in['description'] = "Credits B/F"

del voucher_transactions_in['in']
del voucher_transactions_in['out']


voucher_transactions_out = voucher_trans.copy()
voucher_transactions_out['type'] = 'DEBIT'
voucher_transactions_out['amount'] = voucher_transactions_out['out']
voucher_transactions_out['description'] = "Debits B/F"

del voucher_transactions_out['in']
del voucher_transactions_out['out']


In [13]:
voucher_transactions_in

,created_at,updated_at,type,voucher_id,amount,description
0,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,1,3161.0,Credits B/F
1,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,2,820.0,Credits B/F
2,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,3,0.0,Credits B/F
3,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,4,1144.0,Credits B/F
4,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,5,100.0,Credits B/F
...,...,...,...,...,...,...
819,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,820,0.0,Credits B/F
820,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,821,0.0,Credits B/F
821,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,822,0.0,Credits B/F
822,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,823,0.0,Credits B/F


In [14]:
voucher_transactions_out

,created_at,updated_at,type,voucher_id,amount,description
0,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,DEBIT,1,2110.0,Debits B/F
1,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,DEBIT,2,820.0,Debits B/F
2,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,DEBIT,3,0.0,Debits B/F
3,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,DEBIT,4,1144.0,Debits B/F
4,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,DEBIT,5,100.0,Debits B/F
...,...,...,...,...,...,...
819,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,DEBIT,820,0.0,Debits B/F
820,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,DEBIT,821,0.0,Debits B/F
821,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,DEBIT,822,0.0,Debits B/F
822,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,DEBIT,823,0.0,Debits B/F


In [15]:
voucher_transactions = pd.concat([voucher_transactions_in, voucher_transactions_out]).sort_values('voucher_id')
voucher_transactions

,created_at,updated_at,type,voucher_id,amount,description
0,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,1,3161.0,Credits B/F
0,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,DEBIT,1,2110.0,Debits B/F
1,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,2,820.0,Credits B/F
1,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,DEBIT,2,820.0,Debits B/F
2,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,3,0.0,Credits B/F
...,...,...,...,...,...,...
821,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,822,0.0,Credits B/F
822,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,DEBIT,823,0.0,Debits B/F
822,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,823,0.0,Credits B/F
823,2022-07-26 22:40:32.365141,2022-07-26 22:40:32.365141,CREDIT,824,0.0,Credits B/F


In [16]:
del vouchers_new['in']
del vouchers_new['out']

In [17]:
del stk_requests_new['complete']

In [18]:
stk_callbacks_new[['Phone']] = stk_callbacks[['PhoneNumber']]
del stk_callbacks_new['PhoneNumber']

In [19]:
b2c_result_parameters_new[['transaction_completed_date_time']] = b2c_result_parameters_new[['TransactionCompletedDateTime']]
del b2c_result_parameters_new['TransactionCompletedDateTime']

In [20]:
# Test Data
payableIds = [1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 23,
 27,
 29,
 30,
 31,
 34,
 35,
 36,
 55,
 57,
 59,
 60,
 61,
 85,
 86,
 95,
 101,
 103,
 113,
 114,
 120,
 122,
 123,
 126,
 127,
 128,
 132,
 140,
 142,
 145,
 149,
 153,
 160,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 187,
 194,
 195,
 196,
 197,
 198,
 201,
 207,
 213,
 215,
 237,
 243,
 259,
 265,
 267,
 271,
 279,
 300,
 302,
 308,
 316,
 320,
 322,
 324,
 328,
 329,
 331,
 332,
 335,
 395,
 433,
 438,
 440,
 441,
 442,
 475,
 542,
 543,
 544,
 555,
 563,
 601,
 602,
 634,
 662,
 687,
 732,
 734,
 751,
 772,
 795,
 827,
 864,
 870,
 958,
 975,
 1087,
 1151,
 1310,
 1450,
 1522,
 1608,
 1609,
 1668,
 1699,
 1837,
 1900,
 2103,
 2297,
 2323,
 2415,
 2416,
 2417,
 2419,
 2421,
 2439,
 2453,
 2483,
 2512,
 2577,
 2578,
 2605,
 2650,
 2727,
 2728,
 2729,
 2739,
 2740,
 3011,
 3054,
 3165,
 3365,
 3366,
 3434,
 3705,
 3771,
 3842,
 3883,
 3957,
 3997,
 4115,
 4930,
 5124,
 5214,
 5215,
 5318,
 5362,
 5411,
 5433,
 5502,
 5809,
 5810,
 5838,
 5929,
 5953,
 5956,
 5965,
 6094,
 6153,
 6389,
 6394,
 6516,
 6888,
 6891,
 6927,
 7080,
 7173,
 7179,
 7180,
 7323,
 7470,
 7526,
 7572,
 7655,
 7689,
 7780,
 7834,
 7914,
 7915,
 7916,
 7973,
 7975,
 7998,
 7999,
 8000,
 8024,
 8097,
 8108,
 8111,
 8162,
 8178,
 8182,
 8183,
 8184,
 8185,
 8188,
 8190,
 8193,
 8253,
 8255,
 8280,
 8296,
 8318,
 8351,
 8353,
 8365,
 8386,
 8387,
 8388,
 8389,
 8392,
 8397,
 8409,
 8423,
 8425,
 8447,
 8448,
 8465,
 8505,
 8506,
 8542,
 8566,
 8600,
 8681,
 8711,
 8712,
 8731,
 8827,
 8843,
 8844,
 8846,
 8921,
 8941,
 8979,
 8980,
 8981,
 8982,
 9025,
 9089,
 9103,
 9281,
 9306,
 9344,
 9362,
 9517,
 9552,
 9607,
 9626,
 9698,
 9738,
 9774,
 9820,
 9821,
 9822,
 9853,
 9854,
 9855,
 9896,
 9897,
 9938,
 9990,
 9992,
 10011,
 10125,
 10126,
 10150,
 10151,
 10153,
 10204,
 10232,
 10280,
 10444,
 10493,
 10571,
 10572,
 10573,
 10574,
 10581,
 10716,
 10828,
 10873,
 10890,
 11160,
 11276,
 11278,
 11287,
 11288,
 11291,
 11424,
 11425,
 11428,
 11433,
 11434,
 11435,
 11436,
 11438,
 11439,
 11440,
 11442,
 11445,
 11454,
 11496,
 11500,
 11713,
 11714,
 11721,
 11722,
 11725,
 11726,
 11727,
 11772,
 11773,
 11774,
 11775,
 11776,
 11802,
 11806,
 11810,
 11811,
 11836,
 11853,
 11927,
 11934,
 12049,
 12141,
 12194,
 12244,
 12256,
 12270,
 12291,
 12294,
 12307,
 12308,
 12309,
 12310,
 12339,
 12343,
 12345,
 12367,
 12390,
 12445,
 12504,
 12509,
 12566,
 12586,
 12970,
 12971]

accountIds = [1,2,3,6,7,46]

payments_new = payments_new[payments_new['payable_id'].isin(payableIds)]
product_payments = product_payments[product_payments['payable_id'].isin(payableIds)]
vouchers_new = vouchers_new[vouchers_new['account_id'].isin(accountIds)]

In [21]:
payments_new.to_csv('new/payments/sidooh_payments.csv', index = False)
product_payments.to_csv('new/products/sidooh_payments.csv', index = False)

vouchers_new.to_csv('new/payments/sidooh_vouchers.csv', index = False)
voucher_transactions.to_csv('new/payments/sidooh_voucher_transactions.csv', index = False)

stk_requests_new.to_csv('new/payments/sidooh_mpesa_stk_requests.csv', index = False)
stk_callbacks_new.to_csv('new/payments/sidooh_mpesa_stk_callbacks.csv', index = False)

c2b_callbacks_new.to_csv('new/payments/sidooh_mpesa_c2b_callbacks.csv', index = False)

bulk_requests_new.to_csv('new/payments/sidooh_mpesa_bulk_payment_requests.csv', index = False)
bulk_responses_new.to_csv('new/payments/sidooh_mpesa_bulk_payment_responses.csv', index = False)
b2c_result_parameters_new.to_csv('new/payments/sidooh_mpesa_b2c_result_parameters.csv', index = False)